In [ ]:
# This automates removal of cosmic rays from SDSS red and blue single-epoch spectra,
# as required by Young Sun

# Created 2022 Dec. 3 by E.S.

In [ ]:
'''
Order of operations:

1.) Based on number of single-epoch spectra
    - If 1 spectrum only, ignore for now
    - If 2 spectra only, do a sigma-clipping and identify anomalies *upward* (with a window) 
    - If >=3 spectra, find median spectrum and identify outliers (with a window) 
2.) TBD
'''

In [1]:
import pandas as pd
import numpy as np
import glob
import sys
import os
import matplotlib.pyplot as plt
from astropy.stats import sigma_clip

%matplotlib inline

In [14]:
# find names of spectra for which continuum has been calculated

# top-level directory for SDSS spectra cosmic ray removal
stem_raw_single_epoch = "/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/"+\
                        "01_separated_and_interpolated/"

# find individual file names
file_list_red = glob.glob(stem_raw_single_epoch + "*color_red.csv")
# find all parent names (i.e., one name for each target, whether or not multiepoch observations were made)
parent_list_red = list(set([i.split("_g0")[0] for i in file_list_red]))

In [15]:
parent_list_red

['/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1569-53168-0383',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2297-53738-0361',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2746-54232-0420',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3427-54979-0446',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2796-54629-0416',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1910-53321-0203',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3170-54859-0107',
 '/Users/bandari/Documents/git.rep

In [13]:
for t in range(0,len(parent_list)):
    
    print(t)
    
    # note 'matching' includes both red and blue spectra separately, so '2' matches means 
    # we only have '1' spectrum across all wavelengths
    matching = list(filter(lambda x: parent_list[t] in x, file_list))
    matching_red = list(filter(lambda x: "color_red" in x, matching))
    matching_blue = list(filter(lambda x: "color_blue" in x, matching))
    
    print("-------------------------")
    
    if (len(matching_red) == 1):
        
        print("Only one match across all wavelengths found:")
        print(matching)
    
    elif (len(matching_red) >= 2):
        
        ## ## CONTINUE HERE; NEED TO MAKE THE BELOW RED AND BLUE SPECIFIC, AND TO INDTRODUCE A WINDOW FUNTION
        ## ## (ESP. SINCE NONE OF THESE SPECTRA ARE NORMALIZED)

        # dictionary to hold dataframes
        d = {}
        
        # intialize array to contain all fluxes
        df_dummy = pd.read_csv(matching[0], names=["wavel","flux","noise"], delim_whitespace=True)
        aggregate_flux_array = np.nan*np.ones((len(df_dummy),len(matching)))
        
        # collect spectra in single dictionary
        for p in range(0,len(matching)):
            
            df_single_p = pd.read_csv(matching[p], names=["wavel","flux","noise"], delim_whitespace=True)
            
            #plt.plot(df_single_p["wavel"],df_single_p["flux"])
            
            # sanity check that wavelength abcissa are the same
            if p==0:
                # for checking wavel abcissa is same
                wavel_initial = df_single_p["wavel"].values
            else:
                if len(np.setdiff1d(df_single_p["wavel"].values,wavel_initial) >= 1):
                    print("Hey, the wavelength abcissas are not the same!")
                    sys.exit()

            # put fluxes into array
            aggregate_flux_array[:,p] = df_single_p["flux"].values
            
        # take mean flux of all the spectra
        mean_flux_array = np.mean(aggregate_flux_array,axis=1)
        
        # cast mean spectrum data as DataFrame
        df_mean = pd.DataFrame(mean_flux_array,columns=["avg_flux"])
        df_mean["wavel"] = df_single_p["wavel"] # uses last spectrum read in
        # include median flux too (important for identifying cosmic rays when only 2 spectra are compared)
        median_flux_array = np.median(aggregate_flux_array,axis=1)
        print(median_flux_array)
        df_median = pd.DataFrame(median_flux_array,columns=["median_flux"])
        df_median["wavel"] = df_single_p["wavel"] # uses last spectrum read in
        #mean_flux_array["median_flux"] = pd.Series(median_flux_array.tolist())
        
        for p in range(0,len(matching)):
            # test each empirical spectrum against the mean, and flag points
            df_single_p = pd.read_csv(matching[p], names=["wavel","flux","noise"], delim_whitespace=True)
            flagged_empirical, limit = flag_from_avg(
                                                    df_empir_pass = df_single_p,
                                                    df_avg_pass = df_mean,
                                                    df_median_pass = df_median,
                                                    sigma_choice=5
                                                    )
            
            # if cosmic ray appears to be in an absorption line, discard the spectrum
            ## ## TBD


            #plt.plot(wavel_initial,mean_flux_array,linestyle="--",color="k")
            #plt.show()
            #plt.clf()

            fig = plt.figure(figsize=(24,10))
            plt.plot(flagged_empirical["wavel"],np.subtract(flagged_empirical["flux_flag_1"],1),color="gray",alpha=1)
            #.axvline(x=0, ymin=0, ymax=1
            plt.plot(flagged_empirical["wavel"],flagged_empirical["diff"],label="diff")
            plt.plot(df_mean["wavel"],np.add(df_mean["avg_flux"],0.2),label="mean")
            plt.plot(flagged_empirical["wavel"],flagged_empirical["flux"],label="empirical")
            #plt.plot(df_single_p["wavel"].where(test["flux_flag_1"] == True),
            #             df_single_p["flux"].where(test["flux_flag_1"] == True),
            #         label="flagged",color="k",linewidth=4)
            plt.plot([3900,5000],[limit,limit],linestyle="--")
            plt.title(str(os.path.basename(matching[p])))
            plt.legend(loc="lower right")
            plt.savefig("plot_" + str(os.path.basename(matching[p])) + ".png",
                        facecolor="white", edgecolor='white')
            plt.clf()


0
-------------------------
6 matches found:
['/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1569-53168-0383_g000_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1569-53168-0383_g000_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1569-53168-0383_g001_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1569-53168-0383_g002_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1569-53168-0383_g002_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1569-53168-0383_g001_color_red.csv']
1
-------------

-------------------------
6 matches found:
['/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0998-52750-0023_g000_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0998-52750-0023_g001_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0998-52750-0023_g000_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0998-52750-0023_g002_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0998-52750-0023_g001_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0998-52750-0023_g002_color_red.csv']
39
--------------

-------------------------
10 matches found:
['/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2583-54095-0011_g001_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2583-54095-0011_g002_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2583-54095-0011_g003_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2583-54095-0011_g002_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2583-54095-0011_g004_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2583-54095-0011_g004_color_red.csv', '/Users/bandari/D

-------------------------
6 matches found:
['/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1800-53884-0523_g002_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1800-53884-0523_g001_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1800-53884-0523_g001_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1800-53884-0523_g000_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1800-53884-0523_g002_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1800-53884-0523_g000_color_red.csv']
112
-------------

-------------------------
10 matches found:
['/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3303-54950-0171_g001_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3303-54950-0171_g001_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3303-54950-0171_g000_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3303-54950-0171_g002_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3303-54950-0171_g002_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3303-54950-0171_g003_color_blue.csv', '/Users/bandari

...

-------------------------
16 matches found:
['/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2815-54414-0062_g000_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2815-54414-0062_g004_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2815-54414-0062_g003_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2815-54414-0062_g007_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2815-54414-0062_g004_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-2815-54414-0062_g003_color_blue.csv', '/Users/bandari/D

-------------------------
8 matches found:
['/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0990-52465-0203_g003_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0990-52465-0203_g000_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0990-52465-0203_g003_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0990-52465-0203_g002_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0990-52465-0203_g001_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0990-52465-0203_g000_color_blue.csv', '/Users/bandari/D

-------------------------
8 matches found:
['/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0685-52203-0085_g000_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0685-52203-0085_g002_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0685-52203-0085_g003_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0685-52203-0085_g003_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0685-52203-0085_g001_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-0685-52203-0085_g000_color_blue.csv', '/Users/bandari/

-------------------------
6 matches found:
['/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1719-53876-0294_g000_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1719-53876-0294_g002_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1719-53876-0294_g000_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1719-53876-0294_g001_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1719-53876-0294_g002_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-1719-53876-0294_g001_color_red.csv']
2280
------------

-------------------------
6 matches found:
['/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3311-54943-0071_g000_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3311-54943-0071_g002_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3311-54943-0071_g001_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3311-54943-0071_g001_color_red.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3311-54943-0071_g000_color_blue.csv', '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3311-54943-0071_g002_color_blue.csv']
2317
------------

In [16]:
"color_red" in matching

['/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3005-54876-0348_g001_color_blue.csv',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3005-54876-0348_g003_color_red.csv',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3005-54876-0348_g004_color_red.csv',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3005-54876-0348_g000_color_red.csv',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3005-54876-0348_g000_color_blue.csv',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3005-54876-0348_g002_color_blue.csv',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_fo

In [17]:
matching_red = list(filter(lambda x: "color_red" in x, matching))

['/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3005-54876-0348_g003_color_red.csv',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3005-54876-0348_g004_color_red.csv',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3005-54876-0348_g000_color_red.csv',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3005-54876-0348_g001_color_red.csv',
 '/Users/bandari/Documents/git.repos/rrlfe/notebooks_for_development/sdss_processing/01_separated_and_interpolated/spec-3005-54876-0348_g002_color_red.csv']